In [2]:
import pandas as pd
import numpy
from matplotlib import pyplot as plt
from typing import List, Dict
from collections import Counter
from pprint import pprint

import seaborn as sns
sns.set_style("darkgrid")
sns.set_palette("mako")

import spacy
from spacy.lang.en import English
from nltk.corpus import stopwords

nlp = spacy.load('en_core_web_sm')

pd.set_option('display.float_format', '{:.2f}'.format)


2023-06-24 16:49:13.031488: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [32]:
type(nlp)

spacy.lang.en.English

In [30]:
df_raw = pd.read_parquet("/Users/luis.morales/Desktop/arxiv-paper-recommender/data/processed/arxiv_papers_raw.parquet.gzip")

In [31]:
df_raw.head()

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'created': 'Mon, 2 Apr 2007 19:18:42 GMT', '...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'created': 'Sat, 31 Mar 2007 02:26:18 GMT', ...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'created': 'Sun, 1 Apr 2007 20:46:54 GMT', '...",2008-01-13,"[[Pan, Hongjun, ]]"
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'created': 'Sat, 31 Mar 2007 03:16:14 GMT', ...",2007-05-23,"[[Callan, David, ]]"
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'created': 'Mon, 2 Apr 2007 18:09:58 GMT', '...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"


In [3]:
def cleanData(doc: pd.Series, stemming=False, nlp = spacy.load('en_core_web_sm')):
    """
    TODO: Optimize NLP Object to only obtain stopwords, lemmas, and tokenize docs.
    
    Cleans and processes the input documents by performing various text cleaning operations.

    Args:
        doc (pd.Series): The documents to be cleaned, passed in a Series object.
        stemming (bool, optional): Specifies whether stemming should be applied. Defaults to False.

    Returns:
        str: The cleaned and processed document as a single string.
    """
    doc = doc.lower()
    doc = nlp(doc)
    tokens = [tokens.lower_ for tokens in doc]
    tokens = [tokens for tokens in doc if (tokens.is_stop == False)]
    tokens = [tokens for tokens in tokens if (tokens.is_punct == False)]
    final_token = [token.lemma_ for token in tokens]
    
    return " ".join(final_token)

In [34]:
print(df_raw['abstract'][4])
test= cleanData(df_raw['abstract'][4])
test

  In this paper we show how to compute the $\Lambda_{\alpha}$ norm, $\alpha\ge
0$, using the dyadic grid. This result is a consequence of the description of
the Hardy spaces $H^p(R^N)$ in terms of dyadic and special atoms.



'   paper compute $ \\lambda_{\\alpha}$ norm $ \\alpha\\ge \n 0 $ dyadic grid result consequence description \n hardy space $ h^p(r^n)$ term dyadic special atom \n'

In [35]:
df_raw["cleaned_abstracts"] = df_raw["abstract"].map(lambda x: cleanData(x))

In [36]:

df_raw.to_parquet("/Users/luis.morales/Desktop/arxiv-paper-recommender/data/processed/arxiv_papers.parquet.gzip")